1-Librerias

In [ ]:
pip install tensorflow transformers datasets tensorboard scikit-learn

2- Preprocesamiento de Datos

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Cargar los datos
data = pd.read_csv('tu_archivo.csv')

# Inicializar el tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocesar los textos
def tokenize_texts(texts, tokenizer, max_len=128):
    return tokenizer(
        texts.tolist(),
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

tokenized_texts = tokenize_texts(data['comentario'], tokenizer)

# Convertir etiquetas en clases (para SEGMENTO, PAIS, PERIODO, SENTIMIENTO)
label_encoder_segmento = LabelEncoder()
label_encoder_pais = LabelEncoder()
label_encoder_periodo = LabelEncoder()
label_encoder_sentimiento = LabelEncoder()

data['segmento'] = label_encoder_segmento.fit_transform(data['segmento'])
data['pais'] = label_encoder_pais.fit_transform(data['pais'])
data['periodo'] = label_encoder_periodo.fit_transform(data['periodo'])
data['sentimiento'] = label_encoder_sentimiento.fit_transform(data['sentimiento'])

# Dividir datos en entrenamiento y validación
X_train_texts, X_val_texts, y_train, y_val = train_test_split(tokenized_texts, data[['segmento', 'pais', 'periodo', 'sentimiento']], test_size=0.2)



3- Modelo Multitarea usando BERT

In [ ]:
# Modelo Multitarea usando BERT
def create_multitask_model():
    # Capa BERT para extraer embeddings
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    
    # Entrada de BERT
    input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")
    
    bert_output = bert_model(input_ids, attention_mask=attention_mask)[1]  # Tomamos el embedding del [CLS]
    
    # Capas densas para cada tarea
    sentiment_output = tf.keras.layers.Dense(3, activation='softmax', name='sentiment_output')(bert_output)  # Clasificación de Sentimientos (Positivo, Negativo, Neutro)
    segmento_output = tf.keras.layers.Dense(len(label_encoder_segmento.classes_), activation='softmax', name='segmento_output')(bert_output)
    pais_output = tf.keras.layers.Dense(len(label_encoder_pais.classes_), activation='softmax', name='pais_output')(bert_output)
    periodo_output = tf.keras.layers.Dense(len(label_encoder_periodo.classes_), activation='softmax', name='periodo_output')(bert_output)
    
    # Definir el modelo
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[sentiment_output, segmento_output, pais_output, periodo_output])
    
    # Compilar el modelo con diferentes pérdidas para cada salida
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
        loss={
            'sentiment_output': 'sparse_categorical_crossentropy',
            'segmento_output': 'sparse_categorical_crossentropy',
            'pais_output': 'sparse_categorical_crossentropy',
            'periodo_output': 'sparse_categorical_crossentropy'
        },
        metrics=['accuracy']
    )
    
    return model

# Crear el modelo
model = create_multitask_model()


4- Configuración de Tensorboard

In [ ]:
# Entrenar el modelo
history = model.fit(
    [X_train_texts['input_ids'], X_train_texts['attention_mask']],
    [y_train['sentimiento'], y_train['segmento'], y_train['pais'], y_train['periodo']],
    validation_data=(
        [X_val_texts['input_ids'], X_val_texts['attention_mask']],
        [y_val['sentimiento'], y_val['segmento'], y_val['pais'], y_val['periodo']]
    ),
    epochs=3,
    batch_size=16,
    callbacks=[tensorboard_callback]
)


5- Entrenamiento del Modelo

In [ ]:
# Entrenar el modelo
history = model.fit(
    [X_train_texts['input_ids'], X_train_texts['attention_mask']],
    [y_train['sentimiento'], y_train['segmento'], y_train['pais'], y_train['periodo']],
    validation_data=(
        [X_val_texts['input_ids'], X_val_texts['attention_mask']],
        [y_val['sentimiento'], y_val['segmento'], y_val['pais'], y_val['periodo']]
    ),
    epochs=3,
    batch_size=16,
    callbacks=[tensorboard_callback]
)


In [ ]:
tensorboard --logdir=logs/fit


6- Evaluación y guardado del Modelo

In [ ]:
# Evaluar el modelo
model.evaluate(
    [X_val_texts['input_ids'], X_val_texts['attention_mask']],
    [y_val['sentimiento'], y_val['segmento'], y_val['pais'], y_val['periodo']]
)

# Guardar el modelo entrenado
model.save('modelo_multitarea_bert.h5')


7- Carga y uso del Modelo

In [ ]:
# Cargar el modelo
new_model = tf.keras.models.load_model('modelo_multitarea_bert.h5')
